In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from minio_obj_storage import get_numpy_from_cloud

# Convert dataset order from tensorflow to pytorch. Needed because FZ use tensorflow, while this repo uses Pytorch
tf_2_torch_idx = pd.read_pickle("./mislabelled_exps/tf_id_2_torch_id2.pkl")
tf_2_torch_idx = tf_2_torch_idx['torch_id'].to_numpy()

container_name = 'learning-dynamics-scores'
dataset = 'cifar100'
container_dir = dataset
for arch in ['vgg16', 'mobilenetv2', 'fz_inception']:
    loss = []
    loss_curvature = []
    for epoch in range(200):
        loss_4_eph = get_numpy_from_cloud(container_name, container_dir, f"losses_{dataset}_{arch}_seed_1_epoch_{epoch}_tid0.pt")
        loss_curvature_4_eph = get_numpy_from_cloud(container_name, container_dir, f"loss_curvature_{dataset}_{arch}_seed_1_epoch_{epoch}_h0.001_tid0.pt")
        loss.append(loss_4_eph)
        loss_curvature.append(loss_curvature_4_eph)

    losses = np.array(loss)
    loss_curvatures = np.array(loss_curvature)
    npz = np.load('./analysis_checkpoints/cifar100/cifar100_infl_matrix.npz', allow_pickle=True)
    fz_scores = pd.DataFrame.from_dict({item: npz[item] for item in ['tr_labels', 'tr_mem']})
    fz_scores['loss'] = losses.mean(0)
    fz_scores = fz_scores.sort_values(by='tr_mem', ascending=False)

    print(f"====== {arch} =====")
    for metric in ['loss']:
        all_cs_loss = np.dot(fz_scores['tr_mem'].values, fz_scores[metric].values) / (np.linalg.norm(fz_scores['tr_mem'].values) * np.linalg.norm(fz_scores[metric].values))
        top5k_cs_loss = np.dot(fz_scores['tr_mem'].values[:5000], fz_scores[metric].values[:5000]) / (np.linalg.norm(fz_scores['tr_mem'].values[:5000]) * np.linalg.norm(fz_scores[metric].values[:5000]))
        print(f"All {metric}: {all_cs_loss:.2f}")
        print(f"Top5k {metric}: {top5k_cs_loss:.2f}")